In [3]:
!pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.7 MB/s eta 0:00:00


In [5]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 4.5 MB/s eta 0:00:00


In [13]:
from sklearn.metrics import accuracy_score

In [9]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
import lightgbm as lgb
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras_tuner.tuners import RandomSearch
import optuna

# Load the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)

# Random Forest
rf = RandomForestClassifier()
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}
rf_grid = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=3)
rf_grid.fit(X_train, y_train)
best_rf = rf_grid.best_estimator_
y_pred_rf = best_rf.predict(X_test)

# AdaBoost
ada = AdaBoostClassifier()
param_grid_ada = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1.0],
}
ada_random = RandomizedSearchCV(estimator=ada, param_distributions=param_grid_ada, n_iter=10, cv=3)
ada_random.fit(X_train, y_train)
best_ada = ada_random.best_estimator_
y_pred_ada = best_ada.predict(X_test)

# Gradient Boosting
gb = GradientBoostingClassifier()
param_grid_gb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1.0],
    'max_depth': [3, 5, 7],
}
gb_random = RandomizedSearchCV(estimator=gb, param_distributions=param_grid_gb, n_iter=10, cv=3)
gb_random.fit(X_train, y_train)
best_gb = gb_random.best_estimator_
y_pred_gb = best_gb.predict(X_test)

# Support Vector Machine
svm = SVC()
param_grid_svm = {
    'C': [0.1, 1.0, 10.0],
    'kernel': ['linear', 'rbf'],
}
svm_grid = GridSearchCV(estimator=svm, param_grid=param_grid_svm, cv=3)
svm_grid.fit(X_train, y_train)
best_svm = svm_grid.best_estimator_
y_pred_svm = best_svm.predict(X_test)

# k-Nearest Neighbors
knn = KNeighborsClassifier()
param_grid_knn = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
}
knn_random = RandomizedSearchCV(estimator=knn, param_distributions=param_grid_knn, n_iter=10, cv=3)
knn_random.fit(X_train, y_train)
best_knn = knn_random.best_estimator_
y_pred_knn = best_knn.predict(X_test)

# Naive Bayes
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)

# XGBoost
xgb = XGBClassifier()
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 1.0],
}
xgb_random = RandomizedSearchCV(estimator=xgb, param_distributions=param_grid_xgb, n_iter=10, cv=3)
xgb_random.fit(X_train, y_train)
best_xgb = xgb_random.best_estimator_
y_pred_xgb = best_xgb.predict(X_test)

# LightGBM
lgbm = lgb.LGBMClassifier()
param_grid_lgbm = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 1.0],
}
lgbm_random = RandomizedSearchCV(estimator=lgbm, param_distributions=param_grid_lgbm, n_iter=10, cv=3)
lgbm_random.fit(X_train, y_train)
best_lgbm = lgbm_random.best_estimator_
y_pred_lgbm = best_lgbm.predict(X_test)

# Neural Network
def create_model():
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

nn = KerasClassifier(build_fn=create_model, verbose=0)
param_grid_nn = {
    'batch_size': [16, 32],
    'epochs': [10, 20],
}
nn_random = RandomizedSearchCV(estimator=nn, param_distributions=param_grid_nn, n_iter=4, cv=3)
nn_random.fit(X_train, y_train)
best_nn = nn_random.best_estimator_
y_pred_nn = best_nn.predict(X_test)

# Print the accuracies
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_logreg))
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("AdaBoost Accuracy:", accuracy_score(y_test, y_pred_ada))
print("Gradient Boosting Accuracy:", accuracy_score(y_test, y_pred_gb))
print("Support Vector Machine Accuracy:", accuracy_score(y_test, y_pred_svm))
print("k-Nearest Neighbors Accuracy:", accuracy_score(y_test, y_pred_knn))
print("Naive Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("LightGBM Accuracy:", accuracy_score(y_test, y_pred_lgbm))
print("Neural Network Accuracy:", accuracy_score(y_test, y_pred_nn))


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
<ipython-input-9-e51e110430d9>:130: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  nn = KerasClassifier(build_fn=create_model, verbose=0)


1/1 [==============================] - 0s 51ms/step
Logistic Regression Accuracy: 1.0
Random Forest Accuracy: 1.0
AdaBoost Accuracy: 1.0
Gradient Boosting Accuracy: 1.0
Support Vector Machine Accuracy: 1.0
k-Nearest Neighbors Accuracy: 1.0
Naive Bayes Accuracy: 1.0
XGBoost Accuracy: 1.0
LightGBM Accuracy: 1.0
Neural Network Accuracy: 0.5333333333333333


In [14]:
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
# Evaluation metrics
print("Logistic Regression:")
print("Accuracy:", accuracy_score(y_test, y_pred_logreg))
print("Precision:", precision_score(y_test, y_pred_logreg, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_logreg, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred_logreg, average='weighted'))

print("\nRandom Forest:")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Precision:", precision_score(y_test, y_pred_rf, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_rf, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred_rf, average='weighted'))

print("\nAdaBoost:")
print("Accuracy:", accuracy_score(y_test, y_pred_ada))
print("Precision:", precision_score(y_test, y_pred_ada, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_ada, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred_ada, average='weighted'))

print("\nGradient Boosting:")
print("Accuracy:", accuracy_score(y_test, y_pred_gb))
print("Precision:", precision_score(y_test, y_pred_gb, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_gb, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred_gb, average='weighted'))

print("\nSupport Vector Machine:")
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Precision:", precision_score(y_test, y_pred_svm, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_svm, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred_svm, average='weighted'))

print("\nk-Nearest Neighbors:")
print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("Precision:", precision_score(y_test, y_pred_knn, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_knn, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred_knn, average='weighted'))

print("\nNaive Bayes:")
print("Accuracy:", accuracy_score(y_test, y_pred_nb))
print("Precision:", precision_score(y_test, y_pred_nb, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_nb, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred_nb, average='weighted'))

print("\nXGBoost:")
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("Precision:", precision_score(y_test, y_pred_xgb, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_xgb, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred_xgb, average='weighted'))

print("\nLightGBM:")
print("Accuracy:", accuracy_score(y_test, y_pred_lgbm))
print("Precision:", precision_score(y_test, y_pred_lgbm, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_lgbm, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred_lgbm, average='weighted'))

print("\nNeural Network:")
print("Accuracy:", accuracy_score(y_test, y_pred_nn))
print("Precision:", precision_score(y_test, y_pred_nn, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_nn, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred_nn, average='weighted'))

Logistic Regression:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0

Random Forest:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0

AdaBoost:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0

Gradient Boosting:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0

Support Vector Machine:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0

k-Nearest Neighbors:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0

Naive Bayes:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0

XGBoost:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0

LightGBM:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0

Neural Network:
Accuracy: 0.5333333333333333
Precision: 0.3688888888888889
Recall: 0.5333333333333333
F1-Score: 0.4352564102564102


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
